In [6]:
refSeq = {}
refSeqFile = open('/qbio/prom317/tutorial/references/GRCm39.transcripts.fa.cut', 'r')
lines = refSeqFile.readlines()
transcriptId=""
for line in lines:
    if line.startswith(">"):
        transcriptId = line[1:-1]
        refSeq[transcriptId] = ""
    else:
        refSeq[transcriptId] += line[:-1]

In [7]:
samData={}
samFile = open('/qbio/prom317/tutorial/transCutAlignment/CLIP_35L33G/Aligned.out.sam', 'r')
lines = samFile.readlines()
for line in lines:
    if line.startswith("SRR"):
        linesep = line.split("\t")
        queryName = linesep[0]
        refSeqName = linesep[2]
        pos = int(linesep[3])
        cigar = linesep[5]
        querySeq = linesep[9]
        samData[queryName] = [refSeqName, pos, cigar, querySeq]

In [8]:
import re
for i in samData.keys():
    cigarList=[]
    cigar = samData[i][2]
    while(cigar):
        matchCigar = re.match("[0-9]+[A-Z]+", cigar)
        cigarList.append(matchCigar.group())
        cigar = re.sub(pattern = matchCigar.group(), repl = "", count = 1, string = cigar)
    samData[i][2] = cigarList

In [5]:
deletionData = {}
for i in samData.key():
    samLine = samData[i]
    refPos = samLine[1]
    refSequence = refSeq[samLine[0]]
    querySequence = samLine[3]
    cigar = samLine[2]
    queryCount = 0
    while queryCount <= len(querySequence):
        

['78S', '89M', '7I', '6D', '90M', '32S']


In [25]:
import pysam
samData = pysam.AlignmentFile('/qbio/prom317/tutorial/transCutAlignment/CLIP_35L33G/Aligned.out.sorted.bam', 'rb')
iter = samData.fetch("ENSMUST00000083506.3", 50, 60)
for x in iter:
    if x.query_name == "SRR458758.2.8708628":
        print(str(x.get_aligned_pairs()))

[(0, 33), (1, 34), (2, 35), (3, 36), (4, 37), (5, 38), (6, 39), (7, 40), (8, 41), (9, 42), (10, 43), (11, 44), (12, 45), (13, 46), (14, 47), (15, 48), (16, 49), (17, 50), (18, 51), (19, 52), (20, 53), (21, 54), (22, 55), (23, 56), (24, 57), (25, 58), (26, 59), (27, 60), (28, 61), (29, 62), (30, 63), (31, 64), (32, 65), (33, 66), (34, 67), (35, 68), (36, 69), (37, 70), (38, 71), (39, 72), (40, 73), (41, 74), (42, 75), (43, 76), (44, 77), (45, 78), (46, 79), (47, 80), (48, 81), (49, 82), (50, 83)]
